In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from fedot.api.main import Fedot
import ccxt
import time
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import warnings
from joblib import dump, load
import random as rn
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [4]:
# Define SMAPE for measuring prediction accuracy
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

## Data Collection and Feature Engineering

Gateio daily

In [5]:
# Initialize Gate.io exchange
exchange = ccxt.gateio()
exchange.enableRateLimit = True  # Enable rate limit for safe API calls

# Define the symbol and timeframe
symbol = 'BTC/USDT'  # Symbol for Bitcoin
timeframe = '1d'     # 1 day timeframe
since = exchange.parse8601('2018-01-01T00:00:00Z')  # Start date
until = exchange.parse8601('2024-02-01T00:00:00Z')  # End date


# Function to fetch OHLCV data
def fetch_ohlcv(exchange, symbol, timeframe, since, until):
    ohlcv = []
    limit = 100  # Number of data points per call (can vary based on the exchange)

    while since < until:
        new_ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        if len(new_ohlcv) == 0:
            break
        ohlcv.extend(new_ohlcv)
        since = new_ohlcv[-1][0] + 1  # Update since to get new data
    return ohlcv

# Fetching the data
data = fetch_ohlcv(exchange, symbol, timeframe, since, until)

# Create a DataFrame
df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)
df

,open,high,low,close,volume
timestamp,,,,,
2018-01-01,13588.000000,13745.000148,12786.509982,13300.001043,352.462799
2018-01-02,13300.000000,20000.000000,13000.000000,14520.000000,770.621694
2018-01-03,14563.150000,16014.000000,14200.000000,14802.770000,864.994063
2018-01-04,14727.959999,15341.666667,14100.000000,15050.101039,762.863721
2018-01-05,15050.091670,18261.500000,14746.440000,16900.029998,920.796958
...,...,...,...,...,...
2023-12-30,42070.800000,42613.200000,41521.600000,42149.800000,3485.378282
2023-12-31,42149.800000,42888.800000,41969.400000,42285.400000,3075.964883
2024-01-01,42285.500000,44191.600000,42181.500000,44174.300000,3206.851750


In [2]:
# Correcting columns
col = []
for i in df.columns[:]:
    col.append(i.title())
df.columns = col

In [7]:
def add_technical_indicators(df_):

    data = df_.copy()
    # Add moving averages and other technical indicators
    data['MA_5'] = data['Close'].rolling(window=5).mean()
    data['MA_10'] = data['Close'].rolling(window=10).mean()
    data['MA_20'] = data['Close'].rolling(window=20).mean()

    data['MA_5_1'] = data['MA_5'].shift()
    data['MA_5_change'] = data['MA_5'] - data['MA_5_1']

    data['MA_10_1'] = data['MA_10'].shift()
    data['MA_10_change'] = data['MA_10'] - data['MA_10_1']

    data['MA_20_1'] = data['MA_20'].shift()
    data['MA_20_change'] = data['MA_20'] - data['MA_20_1']


    data['Close_1'] = data.Close.shift()
    data['Close_change'] = data['Close'] - data['Close_1']
    data['Volume_1'] = data.Volume.shift()
    data['Volume_change'] = data['Volume'] - data['Volume_1']

    data['EMA5'] = ta.ema(data.Close, 5)
    data['EMA10'] = ta.ema(data.Close, 10)
    data['EMA20'] = ta.ema(data.Close, 20)

    data['RSI12'] = ta.rsi(data.Close, 12)
    data['OBV'] = ta.obv(data.Close, data.Volume)

    data['ROC5'] = ta.roc(data.Close, 5)
    data['ROC10'] = ta.roc(data.Close, 10)
    data['ROC20'] = ta.roc(data.Close, 20)

    data['MACD'] = ta.macd(data.Close).iloc[:, 0]
    data['MACDhist'] = ta.macd(data.Close).iloc[:, 1]
    data['MACDsignal'] = ta.macd(data.Close).iloc[:, 2]

    data['%K'] = ta.stoch(data.High, data.Low, data.Close).iloc[:, 0]
    data['%D'] = ta.stoch(data.High, data.Low, data.Close).iloc[:, 1]

    data['CCI'] = ta.cci(data.High, data.Low, data.Close)



    return data

In [8]:
def scaler_data(data_):

    """Scaler all columns and return df and scalers dict"""
    df_scaled = data_.copy()

    scalers_ = {}

    for column in df_scaled.columns:
        scaler = StandardScaler()
        df_scaled[column] = scaler.fit_transform(df_scaled[[column]])
        scalers_[column] = scaler
    return df_scaled, scalers_

In [9]:
df_ta = add_technical_indicators(df).dropna()
df_ta

,Open,High,Low,Close,Volume,MA_5,MA_10,MA_20,MA_5_1,MA_5_change,MA_10_1,MA_10_change,MA_20_1,MA_20_change,Close_1,Close_change,Volume_1,Volume_change,EMA5,EMA10,EMA20,RSI12,OBV,ROC5,ROC10,ROC20,MACD,MACDhist,MACDsignal,%K,%D,CCI
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-02-03,8858.780000,10217.000000,8256.88,9231.000249,649.202652,9575.23405,10489.982721,11010.962354,10002.124165,-426.890115,10706.882696,-216.899975,11224.512342,-213.549988,8858.580000,372.420249,1263.286528,-614.083876,9614.502278,10221.775529,11235.364760,33.138876,-43.576599,-18.780166,-19.026314,-31.632349,-1391.505231,-66.070942,-1325.434289,5.966241,5.222932,-142.333041
2018-02-04,9139.670000,11625.500000,8000.00,8195.920000,635.589393,9166.22605,10189.674721,10741.454354,9575.234050,-409.008000,10489.982721,-300.308000,11010.962354,-269.508000,9231.000249,-1035.080249,649.202652,-13.613259,9141.641519,9853.438160,10945.893830,28.247939,-679.165992,-19.969222,-26.815609,-39.674137,-1466.315830,-112.705233,-1353.610597,5.412241,5.295484,-118.588377
2018-02-05,8211.400000,10000.000000,6669.20,7006.000000,1005.286416,8504.62405,9777.657952,10541.482367,9166.226050,-661.602000,10189.674721,-412.016769,10741.454354,-199.971987,8195.920000,-1189.920000,635.589393,369.697023,8429.761013,9335.722131,10570.665846,23.836081,-1684.452408,-32.072976,-37.031328,-36.340572,-1603.140363,-199.623813,-1403.516550,4.116707,5.165063,-146.464963
2018-02-06,7006.000000,8038.001331,6042.50,7714.430000,1968.732890,8201.18605,9405.984952,10378.703867,8504.624050,-303.438000,9777.657952,-371.673000,10541.482367,-162.778500,7006.000000,708.430000,1005.286416,963.446474,8191.317342,9040.941744,10298.643385,30.850497,284.280482,-16.434710,-32.514023,-29.677028,-1635.556731,-185.632145,-1449.924586,4.695739,4.741562,-137.406546
2018-02-07,7681.045607,10344.000000,7150.00,7565.000000,1013.516777,7942.47005,8972.297107,10207.516860,8201.186050,-258.716000,9405.984952,-433.687845,10378.703867,-171.187007,7714.430000,-149.430000,1968.732890,-955.216112,7982.544894,8772.588699,10038.296396,30.210281,-729.236296,-14.602566,-36.438605,-31.156803,-1654.235689,-163.448882,-1490.786807,7.412077,5.408175,-82.215221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-30,42070.800000,42613.200000,41521.60,42149.800000,3485.378282,42545.64000,43083.110000,42683.380000,42830.180000,-284.540000,43235.150000,-152.040000,42765.170000,-81.790000,42070.800000,79.000000,5598.143025,-2112.764743,42510.196055,42718.383990,42480.596272,48.850845,48419.777326,-3.265133,-3.481550,-3.735931,594.586539,-326.970963,921.557502,44.556844,53.224446,-85.790641
2023-12-31,42149.800000,42888.800000,41969.40,42285.400000,3075.964883,42500.18000,42925.490000,42735.010000,42545.640000,-45.460000,43083.110000,-157.620000,42683.380000,51.630000,42149.800000,135.600000,3485.378282,-409.413400,42435.264037,42639.659628,42462.006151,49.785845,51495.742209,-0.534664,-3.593576,2.503103,515.598828,-324.766939,840.365767,42.122965,47.429369,-65.261392
2024-01-01,42285.500000,44191.600000,42181.50,44174.300000,3206.851750,42649.44000,42945.940000,42869.180000,42500.180000,149.260000,42925.490000,20.450000,42735.010000,134.170000,42285.400000,1888.900000,3075.964883,130.886867,43014.942691,42918.685150,42625.081755,60.702231,54702.593959,1.718477,0.465092,6.467442,598.519581,-193.476950,791.996530,59.862377,48.847395,62.123422


## Implementing Regression for next day MA_10 change

In [10]:
# Make a change in MA columns to predict it for test data
# from predicted change in MA, we can determine the price will go up or down
df_ta['Target_MA10'] = df_ta["MA_10"].shift(-1)
df_ta['Target_MA10'] = df_ta['Target_MA10'] - df_ta['MA_10']


# Drop rows with NaN values created by rolling mean and shifting
df_ta.dropna(inplace=True)
df_ta

,Open,High,Low,Close,Volume,MA_5,MA_10,MA_20,MA_5_1,MA_5_change,MA_10_1,MA_10_change,MA_20_1,MA_20_change,Close_1,Close_change,Volume_1,Volume_change,EMA5,EMA10,EMA20,RSI12,OBV,ROC5,ROC10,ROC20,MACD,MACDhist,MACDsignal,%K,%D,CCI,Target_MA10
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-02-03,8858.780000,10217.000000,8256.88,9231.000249,649.202652,9575.23405,10489.982721,11010.962354,10002.124165,-426.890115,10706.882696,-216.899975,11224.512342,-213.549988,8858.580000,372.420249,1263.286528,-614.083876,9614.502278,10221.775529,11235.364760,33.138876,-43.576599,-18.780166,-19.026314,-31.632349,-1391.505231,-66.070942,-1325.434289,5.966241,5.222932,-142.333041,-300.308000
2018-02-04,9139.670000,11625.500000,8000.00,8195.920000,635.589393,9166.22605,10189.674721,10741.454354,9575.234050,-409.008000,10489.982721,-300.308000,11010.962354,-269.508000,9231.000249,-1035.080249,649.202652,-13.613259,9141.641519,9853.438160,10945.893830,28.247939,-679.165992,-19.969222,-26.815609,-39.674137,-1466.315830,-112.705233,-1353.610597,5.412241,5.295484,-118.588377,-412.016769
2018-02-05,8211.400000,10000.000000,6669.20,7006.000000,1005.286416,8504.62405,9777.657952,10541.482367,9166.226050,-661.602000,10189.674721,-412.016769,10741.454354,-199.971987,8195.920000,-1189.920000,635.589393,369.697023,8429.761013,9335.722131,10570.665846,23.836081,-1684.452408,-32.072976,-37.031328,-36.340572,-1603.140363,-199.623813,-1403.516550,4.116707,5.165063,-146.464963,-371.673000
2018-02-06,7006.000000,8038.001331,6042.50,7714.430000,1968.732890,8201.18605,9405.984952,10378.703867,8504.624050,-303.438000,9777.657952,-371.673000,10541.482367,-162.778500,7006.000000,708.430000,1005.286416,963.446474,8191.317342,9040.941744,10298.643385,30.850497,284.280482,-16.434710,-32.514023,-29.677028,-1635.556731,-185.632145,-1449.924586,4.695739,4.741562,-137.406546,-433.687845
2018-02-07,7681.045607,10344.000000,7150.00,7565.000000,1013.516777,7942.47005,8972.297107,10207.516860,8201.186050,-258.716000,9405.984952,-433.687845,10378.703867,-171.187007,7714.430000,-149.430000,1968.732890,-955.216112,7982.544894,8772.588699,10038.296396,30.210281,-729.236296,-14.602566,-36.438605,-31.156803,-1654.235689,-163.448882,-1490.786807,7.412077,5.408175,-82.215221,-307.799082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29,42566.900000,43113.800000,41264.10,42070.800000,5598.143025,42830.18000,43235.150000,42765.170000,43014.280000,-184.100000,43255.880000,-20.730000,42847.445000,-82.275000,42566.900000,-496.100000,5116.950486,481.192539,42690.394082,42844.735988,42515.416932,48.337105,44934.399044,-2.141131,-0.490325,-3.764042,700.561596,-302.738647,1003.300243,55.608297,61.117978,-81.622716,-152.040000
2023-12-30,42070.800000,42613.200000,41521.60,42149.800000,3485.378282,42545.64000,43083.110000,42683.380000,42830.180000,-284.540000,43235.150000,-152.040000,42765.170000,-81.790000,42070.800000,79.000000,5598.143025,-2112.764743,42510.196055,42718.383990,42480.596272,48.850845,48419.777326,-3.265133,-3.481550,-3.735931,594.586539,-326.970963,921.557502,44.556844,53.224446,-85.790641,-157.620000
2023-12-31,42149.800000,42888.800000,41969.40,42285.400000,3075.964883,42500.18000,42925.490000,42735.010000,42545.640000,-45.460000,43083.110000,-157.620000,42683.380000,51.630000,42149.800000,135.600000,3485.378282,-409.413400,42435.264037,42639.659628,42462.006151,49.785845,51495.742209,-0.534664,-3.593576,2.503103,515.598828,-324.766939,840.365767,42.122965,47.429369,-65.261392,20.450000


In [3]:
# Splitiing in test and train and scalering data
df_train, df_test = train_test_split(df_ta, test_size=0.15, shuffle=False)

df_train_scaled, scalers = scaler_data(df_train)

df_test_scaled = df_test.copy()
for i in df_test.columns:
    df_test_scaled[i] = scalers[i].transform(df_test[[i]])

df_train_scaled

In [4]:
# Splitting data into training and test sets
X_train, X_test, y_train, y_test = (df_train_scaled.drop("Target_MA10", axis=1), df_test_scaled.drop("Target_MA10", axis=1),
                                    df_train_scaled['Target_MA10'],  df_test_scaled['Target_MA10'])

In [5]:
# USE Fedot AutoML for solving regression problem

model_regression = Fedot(problem='regression', timeout=30, preset='best_quality', n_jobs=1)
model_regression.fit(features=X_train, target=y_train)

In [150]:
# Validating regression results

y_test_pred = scalers['Target_MA10'].inverse_transform(np.reshape(model_regression.predict(X_test), (-1, 1))).flatten()
y_train_pred = scalers['Target_MA10'].inverse_transform(np.reshape(model_regression.predict(X_train), (-1, 1))).flatten()

y_train_check = scalers['Target_MA10'].inverse_transform(np.reshape(y_train.values , (-1, 1))).flatten()
y_test_check = scalers['Target_MA10'].inverse_transform(np.reshape(y_test.values , (-1, 1))).flatten()


# Calculating metrics
metrics = {
    'Train RMSE': np.sqrt(mean_squared_error(y_train_check, y_train_pred)),
    'Test RMSE': np.sqrt(mean_squared_error(y_test_check, y_test_pred)),
    'Train MAE': mean_absolute_error(y_train_check, y_train_pred),
    'Test MAE': mean_absolute_error(y_test_check, y_test_pred),
    'Train SMAPE': smape(y_train_check, y_train_pred),
    'Test SMAPE': smape(y_test_check, y_test_pred)
}

for metric, value in metrics.items():
    print(f"{metric}: {value}")

Train RMSE: 115.73000515235405
Test RMSE: 77.66627915189592
Train MAE: 65.83977464888898
Test MAE: 55.54545674214571
Train SMAPE: 61.934185927663925
Test SMAPE: 67.29896064629905


In [6]:
# Solve make a predicted changes for train set and create binary variable
# for determining if the price go up or down (signal)
predicted_train = model_regression.predict(X_train)
X_train['Predicted_change'] = predicted_train

# binarize up or down
y_train_cl = np.where(y_train > 0, 1, -1)

y_test_cl = np.where(y_test > 0, 1, -1)

## Implementing classification for getting a signal in price change

In [7]:
# Solve classification task with regression predicted change

model_classif = Fedot(problem='classification', timeout=30, preset='best_quality', n_jobs=-1)
model_classif.fit(features=X_train, target=y_train_cl)

In [8]:
# predicted signals for train sample

y_train_cl_pred = model_classif.predict(X_train)

In [155]:
# apply the predictions for test sample

test_predict_regr = model_regression.predict(X_test)
X_test['Predicted_change'] = test_predict_regr

test_predict_cl = model_classif.predict(X_test)

In [157]:
# Calculating classification metrics
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return accuracy, recall, precision, f1

In [158]:
# Metrics for training set
train_accuracy, train_recall, train_precision, train_f1 = calculate_metrics(y_train_cl, y_train_cl_pred.flatten())

# Metrics for test set
test_accuracy, test_recall, test_precision, test_f1 = calculate_metrics(y_test_cl, test_predict_cl.flatten())

# Print results
print(f"Training Metrics:\nAccuracy: {train_accuracy}\nRecall: {train_recall}\nPrecision: {train_precision}\nF1 Score: {train_f1}\n")
print(f"Test Metrics:\nAccuracy: {test_accuracy}\nRecall: {test_recall}\nPrecision: {test_precision}\nF1 Score: {test_f1}")

Training Metrics:
Accuracy: 0.8850762527233116
Recall: 0.8756815703380589
Precision: 0.8922222222222222
F1 Score: 0.8838745184369841

Test Metrics:
Accuracy: 0.8672839506172839
Recall: 0.8777777777777778
Precision: 0.88268156424581
F1 Score: 0.8802228412256267


Pretty good results for classification. Go on

# Now - Backtesting! Evaluate our signal

Here we simply evaluate our strategies by using simple approach - if we got 1 - we hold and asset until we got -1.

In [34]:
!pip show backtesting

Name: Backtesting
Version: 0.3.3
Summary: Backtest trading strategies in Python
Home-page: https://kernc.github.io/backtesting.py/
Author: Zach Lûster
Author-email: 
License: AGPL-3.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: bokeh, numpy, pandas
Required-by: 


In [35]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [160]:
# form datasets for backtesting
df_initial_train = df.loc[df_train.index].copy()
df_initial_test = df.loc[df_test.index].copy()

df_initial_train

,Open,High,Low,Close,Volume
timestamp,,,,,
2018-02-03,8858.780000,10217.000000,8256.88,9231.000249,649.202652
2018-02-04,9139.670000,11625.500000,8000.00,8195.920000,635.589393
2018-02-05,8211.400000,10000.000000,6669.20,7006.000000,1005.286416
2018-02-06,7006.000000,8038.001331,6042.50,7714.430000,1968.732890
2018-02-07,7681.045607,10344.000000,7150.00,7565.000000,1013.516777
...,...,...,...,...,...
2023-02-08,23247.000000,23442.000000,22671.80,22961.500000,3585.142314
2023-02-09,22962.000000,23006.700000,21700.10,21796.700000,7053.941412
2023-02-10,21796.700000,21929.800000,21455.00,21629.000000,4153.650348


In [164]:

class DynamicModelStrategy(Strategy):
    def init(self):
        # Initialize your strategy
        self.data_counter = 0

    def next(self):
        # Updating the model with new data
        current_data = self.data.df.iloc[:self.data_counter + 1]

        current_data = df_initial_train.append(current_data)
        # Feature engineering and scaling for the new data point
        current_data = add_technical_indicators(current_data).dropna()
        current_data_scaled = current_data[:]
        
        # the part with scaler is not optimized, and it can be optimized for 
        # one operation 
        for i in current_data_scaled.columns:
            current_data_scaled[i] = scalers[i].transform(current_data[[i]])


        prediction_regression = model_regression.predict(current_data_scaled)
        # print('Regression_completeed', prediction_regression.flatten()[-1])

        class_data = current_data_scaled.iloc[-1, :].copy()
        class_data = np.append(class_data.values, prediction_regression.flatten()[-1])
        # Making predictions with the updated model

        signal = model_classif.predict(class_data.reshape((1,-1)))[0][0]
        check_predictions.append([signal, current_data_scaled.index[-1]])

        # Execute trades based on the signal
        if signal > 0 and not self.position.is_long:
            self.buy()
        elif signal < 0 and self.position.is_long:
            self.position.close()


        self.data_counter += 1

In [172]:
bt = Backtest(df_initial_test[:], DynamicModelStrategy, cash=1000000, commission=.0025, margin = 1)
stats = bt.run()
stats

Start                     2023-02-13 00:00:00
End                       2024-01-02 00:00:00
Duration                    323 days 00:00:00
Exposure Time [%]                   60.493827
Equity Final [$]                  1556313.519
Equity Peak [$]                  1680152.9245
Return [%]                          55.631352
Buy & Hold Return [%]              106.457895
Return (Ann.) [%]                   64.590844
Volatility (Ann.) [%]               54.657886
Sharpe Ratio                          1.18173
Sortino Ratio                        3.556914
Calmar Ratio                         3.729882
Max. Drawdown [%]                   -17.31713
Avg. Drawdown [%]                    -4.00398
Max. Drawdown Duration      101 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                   19
Win Rate [%]                        42.105263
Best Trade [%]                      31.230078
Worst Trade [%]                      -5.70192
Avg. Trade [%]                    